
# Two-photon Functional Preprocessing (L395_f10)

This notebook loads repository metadata, derives the preprocessing parameters for a two-photon functional session, and runs the preprocessing pipeline. Output artefacts mirror the existing `02_reg/00_preprocessing/2p_functional` layout, but everything is written into `D:/pipelineTestOutput` for testing.

> Set `REPROCESS = True` if you need to overwrite existing outputs.


In [ ]:
from pathlib import Path
import logging

from social_imaging_scripts.metadata.loader import load_animals
from social_imaging_scripts.preprocessing.two_photon import functional, motion
from social_imaging_scripts.metadata.config import load_project_config
from social_imaging_scripts.metadata.config import resolve_raw_path


logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

In [ ]:

cfg = load_project_config()
PROJECT_ROOT = Path.cwd().parent
ANIMAL_ID = "L395_f10"
OUTPUT_BASE = cfg.output_base_dir  # resolves to /mnt/f/johannes/testoutput on WSL, D:/social_imaging_outputs on Windows

METADATA_BASE = PROJECT_ROOT / "metadata/animals"

FAST_DISK = OUTPUT_BASE  # Path('D:/pipelineTestOutput/temp_suite2p') if you want binaries elsewhere
OPS_PATH = Path("../suite2p_ops_may2025.npy")


In [ ]:
REPROCESS = False # whether to re-run plane splitting if outputs already exist
MOTION_REPROCESS = False
FPS = 2.0  # Hz


ops_template = motion.load_global_ops(OPS_PATH)
ops_template['nonrigid'] = True

In [ ]:

collection = load_animals(base_dir=METADATA_BASE)
animal = collection.by_id(ANIMAL_ID)

functional_sessions = [s for s in animal.sessions if s.session_type == "functional_stack"]
if not functional_sessions:
    raise RuntimeError(f"No functional sessions found for {ANIMAL_ID}")

session = functional_sessions[0]
settings = session.session_data.preprocessing_two_photon
if settings is None:
    raise RuntimeError("Two-photon preprocessing settings missing in metadata")

raw_dir = Path(session.session_data.raw_path)
output_root = OUTPUT_BASE / ANIMAL_ID / "02_reg" / "00_preprocessing" / "2p_functional"

print("Raw directory:", raw_dir)
print("Output root:", output_root)
print("Mode:", settings.mode)
print("Planes:", settings.n_planes)
print("Frames per plane:", settings.frames_per_plane)
print("Blocks:", settings.blocks)


In [ ]:

def preprocess_functional_session(*, animal, session, settings, output_root, reprocess=False):
    output_root = Path(output_root)
    metadata_path = output_root / "01_individualPlanes" / f"{animal.animal_id}_preprocessing_metadata.json"

    if metadata_path.exists() and not reprocess:
        print(f"Skipping preprocessing for {animal.animal_id} (metadata exists at {metadata_path})")
        return {"metadata": metadata_path}

    results = functional.run(
        animal_id=animal.animal_id,
        session_id=session.session_id,
        raw_dir=resolve_raw_path(Path(animal.root_dir) / session.session_data.raw_path),
        output_root=output_root,
        settings=settings,
    )
    print("Generated artefacts:")
    for key, path in results.items():
        print(f"  {key}: {path}")
    return results


In [ ]:

results = preprocess_functional_session(
    animal=animal,
    session=session,
    settings=settings,
    output_root=output_root,
    reprocess=REPROCESS,
)


In [ ]:
plane_dir = output_root / '01_individualPlanes'
plane_tiffs = sorted(plane_dir.glob(f'{ANIMAL_ID}_plane*.tif'))
if not plane_tiffs:
    raise FileNotFoundError(f'No plane TIFFs found in {plane_dir}')

motion_results = []
for plane_path in plane_tiffs:
    try:
        plane_idx = int(plane_path.stem.split('plane')[-1])
    except ValueError as err:
        raise ValueError(f'Unexpected plane filename: {plane_path.name}') from err

    res = motion.run_motion_correction(
        animal=animal,
        plane_idx=plane_idx,
        plane_tiff=plane_path,
        ops_template=ops_template,
        fps=FPS,
        output_root=output_root,
        fast_disk=FAST_DISK,
        reprocess=MOTION_REPROCESS,
    )
    motion_results.append(res)
    print(f'Processed plane {plane_idx}')

In [ ]:
print('Motion correction outputs:')
for res in motion_results:
    for key, path in res.items():
        print(f'  {key}: {path}')

In [ ]:

from itertools import islice

plane_dir = output_root / "01_individualPlanes"
if plane_dir.exists():
    print("Sample outputs:")
    for path in islice(sorted(plane_dir.glob("*.tif")), 5):
        print("  ", path)
    metadata_file = plane_dir / f"{animal.animal_id}_preprocessing_metadata.json"
    if metadata_file.exists():
        print("Metadata:", metadata_file)


## FireANTs registration test

In [ ]:
from social_imaging_scripts.preprocessing.two_photon import anatomy as anatomy_preproc
try:
    from social_imaging_scripts.registration.fireants import FireANTsConfig, register_two_photon_anatomy
except ImportError as exc:
    fireants_available = False
    print("FireANTs integration unavailable:", exc)
else:
    fireants_available = True

REFERENCE_BRAIN_RELATIVE = Path("03_Common_Use/reference brains/ref_05_LB_Perrino_2p/average_2p_noRot_orig.nrrd")
reference_brain_path = resolve_raw_path(REFERENCE_BRAIN_RELATIVE, cfg=cfg)

In [ ]:
ANATOMY_REPROCESS = False

anatomy_sessions = [
    s for s in animal.sessions
    if s.session_type == "anatomy_stack" and getattr(s.session_data, "stack_type", "") == "two_photon"
]
if not anatomy_sessions:
    raise RuntimeError(f"No two-photon anatomy session found for {ANIMAL_ID}")

anatomy_session = anatomy_sessions[0]
raw_anatomy = Path(animal.root_dir) / anatomy_session.session_data.raw_path
raw_anatomy_full = resolve_raw_path(raw_anatomy, cfg=cfg)
raw_anatomy_dir = raw_anatomy_full if raw_anatomy_full.is_dir() else raw_anatomy_full.parent

ANATOMY_OUTPUT_ROOT = OUTPUT_BASE / ANIMAL_ID / "02_reg" / "00_preprocessing" / "2p_anatomy"
ANATOMY_OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

anatomy_stack_path = ANATOMY_OUTPUT_ROOT / f"{ANIMAL_ID}_anatomy_stack.tif"
if ANATOMY_REPROCESS or not anatomy_stack_path.exists():
    print("Running anatomy preprocessing...")
    anatomy_preproc.run(
        animal_id=animal.animal_id,
        session_id=anatomy_session.session_id,
        raw_dir=raw_anatomy_dir,
        output_root=ANATOMY_OUTPUT_ROOT,
        settings=anatomy_session.session_data.preprocessing_two_photon,
    )
else:
    print("Using cached anatomy stack:", anatomy_stack_path)

In [ ]:
if not anatomy_stack_path.exists():
    raise FileNotFoundError(f"Anatomy stack not found at {anatomy_stack_path}")

if not fireants_available:
    print("FireANTs module unavailable; skipping registration test.")
else:
    registration_output_root = OUTPUT_BASE / ANIMAL_ID / "02_reg" / "01_fireants"
    registration_output_root.mkdir(parents=True, exist_ok=True)

    fireants_cfg = FireANTsConfig()

    registration_outputs = register_two_photon_anatomy(
        animal_id=ANIMAL_ID,
        session_id=anatomy_session.session_id,
        stack_path=anatomy_stack_path,
        output_root=registration_output_root,
        reference_brain_path=reference_brain_path,
        config=fireants_cfg,
    )
    print("Warped stack:", registration_outputs.get("warped_stack"))
    print("QC image:", registration_outputs.get("qc_figure"))

    qc_path = registration_outputs.get("qc_figure")
    if qc_path:
        from IPython.display import Image, display
        display(Image(filename=qc_path))